In [1]:
# basic
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from datetime import date

# internet
import urllib3
import certifi
import xmlrpc.client

# system / unix
import sys
import os
import re
import multiprocessing
import gzip as gz
import subprocess
from subprocess import call
from subprocess import Popen, PIPE
import uuid
import hashlib
import cProfile
import time
from numba import jit
import shutil

# dataframe and serialization
import pandas as pd
import json
import collections
from collections import deque,OrderedDict
from functools import reduce

# NextGen
# parsing bam
import pysam
# liftover
from pyliftover import LiftOver
# parsing vcf
import vcf as pyvcf
from vcf.parser import _Info as VcfInfo, field_counts as vcf_field_counts

# Graphics
import seaborn as sns

# Stats
import sklearn
from sklearn import linear_model
from scipy.spatial.distance import pdist, squareform
from scipy.stats.stats import pearsonr

#pd.set_option('display.height', 1300)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth',1000)
%matplotlib inline
%autosave 0

Autosave disabled


In [241]:
H_dict={
    ('AA','UU'):-6.82,
    ('AU','AU'):-9.38,
    ('UA','UA'):-7.69,
    ('CU','AG'):-10.48,
    ('CA','UG'):-10.44,
    ('GU','AC'):-11.40,
    ('GA','UC'):-12.44,
    ('CG','CG'):-10.64,
    ('GG','CC'):-13.39,
    ('GC','GC'):-14.88,
    'Init': 3.61,
    'tAU': 3.72,
    'symSelf':0,
}
S_dict={
    ('AA','UU'):-19.0,
    ('AU','AU'):-26.7,
    ('UA','UA'):-20.5,
    ('CU','AG'):-27.1,
    ('CA','UG'):-26.9,
    ('GU','AC'):-29.5,
    ('GA','UC'):-32.5,
    ('CG','CG'):-26.7,
    ('GG','CC'):-32.7,
    ('GC','GC'):-36.9,
    'Init': -1.5,
    'tAU': 10.5,
    'symSelf':-1.4,
}
G_37_1M_7_dict={
    ('AA','UU'):-0.93,
    ('AU','AU'):-1.10,
    ('UA','UA'):-1.33,
    ('CU','AG'):-2.08,
    ('CA','UG'):-2.11,
    ('GU','AC'):-2.24,
    ('GA','UC'):-2.35,
    ('CG','CG'):-2.36,
    ('GG','CC'):-3.26,
    ('GC','GC'):-3.42,
    'Init': 4.09,
    'tAU': 0.45,
    'symSelf': 0.43,
}
rc_dict={
    'AA':'UU',
    'AU':'AU',
    'UA':'UA',
    'CU':'AG',
    'CA':'UG',
    'GU':'AC',
    'GA':'UC',
    'CG':'CG',
    'GG':'CC',
    'GC':'GC',
    'UU':'AA',
    'AG':'CU',
    'UG':'CA',
    'AC':'GU',
    'UC':'GA',
    'CC':'GG'
}
GU_dict={
    ('UU','GG'):-1.10,
    ('GU','GU'):-1.10,
    ('UG','UG'):-1.10,
    ('GG','UU'):-1.10,
        }
revcomp_dict={
    'A':'U',
    'C':'G',
    'G':'C',
    'U':'A',
    'T':'A'
}

In [18]:
# this function unfolds the tuples in the dictionary above for convenience in computation
def unfoldDict(indict):
    outdict={}
    for k in indict:
        v=indict[k]
        if isinstance(k,tuple):
            for u in k:
                outdict[u]=v
        else:
            outdict[k]=v
    return outdict

In [88]:
#5'->3'
s_dict=unfoldDict(S_dict)
h_dict=unfoldDict(H_dict)
g_dict=unfoldDict(G_37_1M_7_dict)

In [89]:
# Chen and Znosko, 2013
def fgc(seq):
    n=0
    nGC=0
    for c in seq:
        n+=1
        if c=='G' or c=='C':
            nGC+=1
    fgc=float(nGC)/float(n)
    return fgc
    
def saltCorrectG(dG,seq,**kwargs):
    na1=kwargs.get('NA1',1.0)
    na2=kwargs.get('NA2',0.122)
#    print(na1,na2)
    q0=fgc(seq)
    q1=np.log(na2/na1)
    q2=np.log(na2)**2
    q3=np.log(na1)**2
#    print(dG,q1,q2,q3)
    G=dG + (0.324*q0 -0.468)*q1 + 0.133*(q2-q3)
    return G
# paper calculation validation
saltCorrectG(-8.62,"AAGUGAUC",NA1=1.021,NA2=0.122)

-7.295294584688707

In [73]:
siRNA_df=pd.read_excel('/nfs/home/mcgold/data/Wuhan-Hu-1/Wuhan_specific_21mers_nodimers.xlsx')

In [205]:
gencode33_21_df=pd.read_csv('/nfs/home/mcgold/data/Wuhan-Hu-1/gencode.v33.transcripts_21mers_sorted.tsv',sep="\t")

NameError: name 'gencode33_21' is not defined

In [96]:
siRNA_dict={}
for i in siRNA_df.index:
    seq=siRNA_df.loc[i,'seq']
    try:
        if len(seq)>=21:
            print(seq)
            siRNA_dict[seq]=[0,0]
    except:
        pass

ACACTACAGCTGATGCATACA
ACTACAGCTGATGCATACATA
ATGTATGCATCAGCTGTAGTG
CTGTAGCGACTGTATGCAGCA
ACTAGTACTGATGTCGTATAC
CTAGTACTGATGTCGTATACA
CTGTATACGACATCAGTACTA
AGCTGTATACACTATGCGAGC
ATAGTACTACAGATAGAGACA
CATAGTACTACAGATAGAGAC
CGACGACGACTACTAGCGTGC
CTCTATCTGTAGTACTATGAC
CTGCTCGCATAGTGTATACAG
GCTGTATACACTATGCGAGCA
GTGTCTCTATCTGTAGTACTA
TCATAGTACTACAGATAGAGA
TCTGCTCGCATAGTGTATACA
AGTCACATCTGTGACATCACA
ACTCGTGACATAGCATCTACA
CTCGTGACATAGCATCTACAG
ATACGACATCAGTACTAGTGC
ATCGACATAGCGAGTGTATGC


In [97]:
siRNA_dict

{'ACACTACAGCTGATGCATACA': [0, 0],
 'ACTACAGCTGATGCATACATA': [0, 0],
 'ATGTATGCATCAGCTGTAGTG': [0, 0],
 'CTGTAGCGACTGTATGCAGCA': [0, 0],
 'ACTAGTACTGATGTCGTATAC': [0, 0],
 'CTAGTACTGATGTCGTATACA': [0, 0],
 'CTGTATACGACATCAGTACTA': [0, 0],
 'AGCTGTATACACTATGCGAGC': [0, 0],
 'ATAGTACTACAGATAGAGACA': [0, 0],
 'CATAGTACTACAGATAGAGAC': [0, 0],
 'CGACGACGACTACTAGCGTGC': [0, 0],
 'CTCTATCTGTAGTACTATGAC': [0, 0],
 'CTGCTCGCATAGTGTATACAG': [0, 0],
 'GCTGTATACACTATGCGAGCA': [0, 0],
 'GTGTCTCTATCTGTAGTACTA': [0, 0],
 'TCATAGTACTACAGATAGAGA': [0, 0],
 'TCTGCTCGCATAGTGTATACA': [0, 0],
 'AGTCACATCTGTGACATCACA': [0, 0],
 'ACTCGTGACATAGCATCTACA': [0, 0],
 'CTCGTGACATAGCATCTACAG': [0, 0],
 'ATACGACATCAGTACTAGTGC': [0, 0],
 'ATCGACATAGCGAGTGTATGC': [0, 0]}

In [123]:
map(lambda x: )

-0.93
C C


In [169]:
def NN_0(Q):
    pos=0
    Qlen=len(Q)
    Q=Q.replace('T', 'U')
    print(Q)
# gu mismatches are treated as AU matches
#Biochemistry. 1986 Jun 3;25(11):3209-13.
#Free energy contributions of G.U and other terminal mismatches to helix stability.
#Q:GU S:GU == AU AU
    nQ = Qlen -1
    dG_QQ=0
    if int(nQ/2) < Qlen/2:
        nQ+=1
    while pos < nQ-1:
        nnQ=str(Q[pos])+str(Q[pos+1])
        try:
            dG_QQ=dG_QQ + g_dict[nnQ]
        except:
            pass
        pos+=1
    return(dG_QQ)

In [210]:
def NN(Q,S):
    #Q and S have to be equal length
    pos=0
    Qlen=len(Q)
    Slen=len(S)
    Q=Q.replace('T', 'U')
    S=S.replace('T', 'U')
# gu mismatches are treated as AU matches
#Biochemistry. 1986 Jun 3;25(11):3209-13.
#Free energy contributions of G.U and other terminal mismatches to helix stability.
#Q:GU S:GU == AU AU
    nQ = Qlen -1
    dG_QS=0
    if int(nQ/2) < Qlen/2:
        nQ+=1
    nS = Slen -1
    if int(nS/2) < Slen/2:
        nS+=1
    while pos < nQ-1:
        nnQ=Q[pos]+str(Q[pos+1])
        nnS=str(S[pos])+str(S[pos+1])
        pos+=1
        try:
            rc_nnS=rc_dict[nnS]
        except:
            rc_nnS='.'
            print(nnS)
        if rc_nnS==nnQ:
            dG_QS=dG_QS+g_dict[nnQ]
        try:
            if (nnQ,rc_nnS) in GU_dict:
                dG_QS=dG_QS+GU_dict[(nnQ,rc_nnS)]
        except:
            pass
    return(dG_QS)

In [186]:
NN_0('GTGTCTCTATCTGTAGTACTA')

GUGUCUCUAUCUGUAGUACUA


-39.29

In [191]:
def rna_rc(seq):
    rcSeq=''
    for i in seq:
        rcSeq=rcSeq + revcomp_dict[i]
    return rcSeq[::-1]
        

In [222]:
siRNA_df=pd.DataFrame.from_dict(siRNA_dict,orient='index')
siRNA_df.columns=['dG_virus','dG_host']

In [223]:
siRNA_df['dG_virus']=siRNA_df.index.map(lambda x: NN_0(x))

ACACUACAGCUGAUGCAUACA
ACUACAGCUGAUGCAUACAUA
AUGUAUGCAUCAGCUGUAGUG
CUGUAGCGACUGUAUGCAGCA
ACUAGUACUGAUGUCGUAUAC
CUAGUACUGAUGUCGUAUACA
CUGUAUACGACAUCAGUACUA
AGCUGUAUACACUAUGCGAGC
AUAGUACUACAGAUAGAGACA
CAUAGUACUACAGAUAGAGAC
CGACGACGACUACUAGCGUGC
CUCUAUCUGUAGUACUAUGAC
CUGCUCGCAUAGUGUAUACAG
GCUGUAUACACUAUGCGAGCA
GUGUCUCUAUCUGUAGUACUA
UCAUAGUACUACAGAUAGAGA
UCUGCUCGCAUAGUGUAUACA
AGUCACAUCUGUGACAUCACA
ACUCGUGACAUAGCAUCUACA
CUCGUGACAUAGCAUCUACAG
AUACGACAUCAGUACUAGUGC
AUCGACAUAGCGAGUGUAUGC


In [224]:
rna_rc('AUCGACAUAGCGAGUGUAUGC')

'GCAUACACUCGCUAUGUCGAU'

In [225]:
siRNA_df['RNAseq_F-OH']=siRNA_df.index.map(lambda x: x.replace('T','U'))
siRNA_df['RNAseq_R-OH']=siRNA_df['RNAseq_F-OH'].map(lambda x: rna_rc(x))
siRNA_df['RNAseqX_host-OH']='.'

In [234]:
siRNA_df['Tm_virus']

,dG_virus,dG_host,RNAseq_F-OH,RNAseq_R-OH,RNAseqX_host-OH,ddG
ACACTACAGCTGATGCATACA,-41.91,-27.72,ACACUACAGCUGAUGCAUACA,UGUAUGCAUCAGCUGUAGUGU,AACAGGCAUACGCCACCACAC,14.19
ACTACAGCTGATGCATACATA,-39.99,-26.27,ACUACAGCUGAUGCAUACAUA,UAUGUAUGCAUCAGCUGUAGU,ACCACACAUGGAUGCCACCUA,13.72
ATGTATGCATCAGCTGTAGTG,-40.77,-28.60,AUGUAUGCAUCAGCUGUAGUG,CACUACAGCUGAUGCAUACAU,ACACCAGCCAUGGCCACCACA,12.17
CTGTAGCGACTGTATGCAGCA,-44.32,-26.65,CUGUAGCGACUGUAUGCAGCA,UGCUGCAUACAGUCGCUACAG,ACACAGCGGUCACCAGUGGCA,17.67
ACTAGTACTGATGTCGTATAC,-38.48,-22.02,ACUAGUACUGAUGUCGUAUAC,GUAUACGACAUCAGUACUAGU,ACAUGUGGUGAUGUUGUAGAC,16.46
CTAGTACTGATGTCGTATACA,-38.35,-23.03,CUAGUACUGAUGUCGUAUACA,UGUAUACGACAUCAGUACUAG,AUCACGUCACCACGACUUGUG,15.32
CTGTATACGACATCAGTACTA,-38.35,-20.98,CUGUAUACGACAUCAGUACUA,UAGUACUGAUGUCGUAUACAG,AAACAGCAUCACCACAUAUGU,17.37
AGCTGTATACACTATGCGAGC,-42.53,-24.52,AGCUGUAUACACUAUGCGAGC,GCUCGCAUAGUGUAUACAGCU,AAGAAAAUAACACGUGUGGCC,18.01
ATAGTACTACAGATAGAGACA,-38.15,-23.41,AUAGUACUACAGAUAGAGACA,UGUCUCUAUCUGUAGUACUAU,AUAACCACGUCUCUCUCUCUG,14.74
CATAGTACTACAGATAGAGAC,-38.15,-26.41,CAUAGUACUACAGAUAGAGAC,GUCUCUAUCUGUAGUACUAUG,ACAGAGCGAGACUACGUCUCA,11.74


In [ ]:
# method one brute force

In [261]:
def displace(Q,Qrc):
    reX=''
    reXrc=''
    for bp in Q:
        if bp in ['A','C']:
            reX=reX+bp
        else:
            reX=reX+'[ACGTU]'
    for bp in Qrc:
        if bp in ['A','C']:
            reXrc=reXrc + bp
        else:
            reXrc=bp + '[ACGTU]' + reXrc
    return (reX,reXrc)
        

In [ ]:
# method2 differential displacement
# 1 select all Host kmers ignoring G mismatches
hostFile='gencode.v33.transcripts_21mers_sorted.tsv'
hostPath=os.path.join('/nfs/home/mcgold/data/Wuhan-Hu-1',hostFile)
siRNA_df['dGx_host']=0;
for i in siRNA_df.index:
    Q=siRNA_df.loc[i,'RNAseq_F-OH']
    Qrc=siRNA_df.loc[i,'RNAseq_R-OH']
    min_dG2_host=0
    reX,reXrc=displace(Q,Qrc)
    sys.stdout.write("Checking \n\'" + reX + "|" + reXrc + "\'...\n")
# only check displaced oligos
    cmd='grep -P \'' + reX + "|" + reXrc + "\' " + hostPath + " > /tmp/mcgold/X.txt"
    subprocess.call(cmd, shell=True)
    cmd='cat /tmp/mcgold/X.txt >> /tmp/mcgold/Xtargets.tsv'
    subprocess.call(cmd, shell=True)
    for line in f:
        seq=line.split("\t")[0]
        seq=seq.strip()
        seq=seq.replace('T', 'U')
        rcSeq=rna_rc(seq)

        for i in siRNA_df.index:
            Q=siRNA_df.loc[i,'RNAseq_F-OH']
            Qrc=siRNA_df.loc[i,'RNAseq_R-OH']
            min_dG_host2=siRNA_df.loc[i,'dG_host']
            q1=NN(Q,seq)
            q2=NN(Qrc,seq)
            q3=NN(Q,rcSeq)
            q4=NN(Qrc,seq)
            qmin=min(q1,q2,q3,q4)
            if qmin < min_dG2_host:
                siRNA_df.loc[i,'dGx_host']=qmin
                siRNA_df.loc[i,'RNAseqX_host-OH']=seq
                sys.stdout.write("Xseq found " + seq + "\n")
    f.close()

Checking 
'ACAC[ACGTU]ACA[ACGTU]C[ACGTU][ACGTU]A[ACGTU][ACGTU]CA[ACGTU]ACA|U[ACGTU]G[ACGTU]U[ACGTU]G[ACGTU]U[ACGTU]G[ACGTU]U[ACGTU]G[ACGTU]U[ACGTU]G[ACGTU]U[ACGTU]U[ACGTU]G[ACGTU]U[ACGTU]ACACACA'...


In [ ]:
siRNA_df['ddG']=siRNA_df['dGx_host']-siRNA_df['dG_virus']

In [ ]:
siRNA_df.to_csv('/nfs/home/mcgold/data/Wuhan-Hu-1/siRNA_df.tsv',sep="\t",header=True)